In [1]:
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import time
from torchsummary import summary
import config
from facenet_pytorch import training, InceptionResnetV1
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from PIL import Image
import glob
import random
import tqdm
from sklearn.metrics import roc_auc_score

In [2]:
class CelebA(data.Dataset):
    def __init__(self, data_path, attr_path, image_size, mode, selected_attrs):
        super(CelebA, self).__init__()
        self.data_path = data_path
        att_list = open(attr_path, 'r', encoding='utf-8').readlines()[1].split()
        atts = [att_list.index(att) + 1 for att in selected_attrs]
        images = np.loadtxt(attr_path, skiprows=2, usecols=[0], dtype=np.str)
        labels = np.loadtxt(attr_path, skiprows=2, usecols=atts, dtype=np.int)
        
        self.tf = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        self.tf_a = transforms.Compose([
            transforms.RandomAffine(degrees=(3, 3),translate=(0.1, 0.1),scale=(0.9, 1.1)),
            transforms.ColorJitter(hue=.05, saturation=.05),
        ])        
        if mode == 'train':
            self.images = images[:182000]
            self.labels = labels[:182000]

        if mode == 'valid':
            self.images = images[182000:182637]
            self.labels = labels[182000:182637]

        if mode == 'test':
            self.images = images[182637:]
            self.labels = labels[182637:]
                                       
        self.length = len(self.images)
    def __getitem__(self, index):
        if index < 182000:
            if random.random() > 0.5:
                img = self.tf(self.tf_a(Image.open(os.path.join(self.data_path, self.images[index]))))
            else:
                img = self.tf(Image.open(os.path.join(self.data_path, self.images[index])))
        else:
            img = self.tf(Image.open(os.path.join(self.data_path, self.images[index])))
        att = torch.tensor((self.labels[index] + 1) // 2)
        return img, att.to(torch.float32)
    def __len__(self):
        return self.length

In [3]:
class CelebA_extra(data.Dataset):
    def __init__(self, data_path, attr_path, image_size, selected_attrs):
        super(CelebA_extra, self).__init__()
        self.data_path = data_path
        att_list = open(attr_path, 'r', encoding='utf-8').readlines()[1].split()
        images = np.genfromtxt(attr_path, skip_header=0, usecols=[0], dtype=np.str)
        labels = np.genfromtxt(attr_path, skip_header=0, usecols=range(1,41), dtype=np.float32)
        
        self.images = images
        self.labels = labels
        self.tf = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        self.tf_a = transforms.Compose([
            transforms.RandomAffine(degrees=(3, 3),translate=(0.1, 0.1),scale=(0.9, 1.1)),
            transforms.ColorJitter(hue=.05, saturation=.05),
        ])   
                                       
        self.length = len(self.images)
    def __getitem__(self, index):
        if random.random() > 0.5:
            img = self.tf(self.tf_a(Image.open(os.path.join(self.data_path, self.images[index]))))
        else:
            img = self.tf(Image.open(os.path.join(self.data_path, self.images[index])))
            
        att = torch.tensor((self.labels[index] + 1) / 2)
        return img, att.to(torch.float32)
    def __len__(self):
        return self.length

In [4]:
attrs_default = ["5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin", "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie", "Young"]

In [5]:
# Root directory for dataset
data_root = "/home/mehmetyavuz/datasets/YFCC100m/img_align_celeba_standardized/"
data_root_extra = "/home/mehmetyavuz/datasets/YFCC100m/"
attr_root = "/home/mehmetyavuz/datasets/CelebA/list_attr_celeba.txt"
attr_root_extra = "labels_filtered_train.txt"
#data_path  = "/home/mehmetyavuz/datasets/YFCC100m/list_of_face_images.txt"
# Number of workers for dataloader
workers = os.cpu_count()

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = (128,128)
epochs = 32

### Classifier

In [6]:
dataset = CelebA(data_root, attr_root, image_size, "train", attrs_default)
dataset_extra = CelebA_extra(data_root_extra, attr_root_extra, image_size, attrs_default)
train_loader = torch.utils.data.DataLoader(dataset,
#train_loader = torch.utils.data.DataLoader(dataset_extra,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=workers)
dataset = CelebA(data_root, "/home/mehmetyavuz/datasets/CelebA/list_attr_celeba.txt", image_size, 'valid',attrs_default)
val_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=workers)
dataset = CelebA(data_root, "/home/mehmetyavuz/datasets/CelebA/list_attr_celeba.txt", image_size, 'test',attrs_default)
test_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=workers)

In [7]:
device = torch.device("cuda:0")

In [8]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=40
).to(device)

In [9]:
resnet = nn.DataParallel(resnet)

In [10]:
resnet.load_state_dict(torch.load("resnet_fe_resnet50_feature_outlier_pretraining*.pth", map_location=device))

<All keys matched successfully>

In [11]:
optimizer = optim.Adam(resnet.parameters(), lr=0.00001)
scheduler = MultiStepLR(optimizer, [20, 21])

In [12]:
loss_fn = torch.nn.BCEWithLogitsLoss()
metrics = {
    'acc': training.accuracy
} 

In [13]:
print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, test_loader,
    batch_metrics=metrics, show_running=True, device=device,
    #writer=writer
)

val_loss = 1
for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        #writer=writer
    )

    resnet.eval()
    val_metrics = training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        #writer=writer
    )
    
    if val_metrics[0].item() < val_loss:
        val_loss = val_metrics[0].item()
        print('Test set Accuracy Lowest Validation Loss:')
        training.pass_epoch(
            resnet, loss_fn, test_loader,
            batch_metrics=metrics, show_running=True, device=device,
            #writer=writer
        )
        torch.save(resnet.state_dict(), "resnet_fe_resnet50_feature_outlier*.pth")



Initial
----------
Valid |   156/156  | loss:    0.2396 | acc:    0.9113   

Epoch 1/32
----------
Train |  1422/1422 | loss:    0.2368 | acc:    0.9110   
Valid |     5/5    | loss:    0.1937 | acc:    0.9210   
Test set Accuracy Lowest Validation Loss:
Valid |   156/156  | loss:    0.2186 | acc:    0.9118   

Epoch 2/32
----------
Train |  1422/1422 | loss:    0.2068 | acc:    0.9156   
Valid |     5/5    | loss:    0.1874 | acc:    0.9226   
Test set Accuracy Lowest Validation Loss:
Valid |   156/156  | loss:    0.2084 | acc:    0.9131   

Epoch 3/32
----------
Train |  1422/1422 | loss:    0.1976 | acc:    0.9168   
Valid |     5/5    | loss:    0.1827 | acc:    0.9239   
Test set Accuracy Lowest Validation Loss:
Valid |   156/156  | loss:    0.2018 | acc:    0.9139   

Epoch 4/32
----------
Train |  1422/1422 | loss:    0.1921 | acc:    0.9180   
Valid |     5/5    | loss:    0.1782 | acc:    0.9248   
Test set Accuracy Lowest Validation Loss:
Valid |   156/156  | loss:    0.199

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f11c9017160>>
Traceback (most recent call last):
  File "/home/mehmetyavuz/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/mehmetyavuz/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1297, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 47, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [14]:
resnet.load_state_dict(torch.load("resnet_fe_resnet50_feature_outlier*.pth", map_location=device))

<All keys matched successfully>

In [15]:
resnet.eval()
counter = 0
for i, batch in enumerate(test_loader):
    if i == 0:
        y_pred = (nn.Sigmoid()(resnet(batch[0].to(device)))).detach().cpu().numpy()
        label = batch[1]
    else:
        y_pred = np.concatenate((y_pred,nn.Sigmoid()(resnet(batch[0].to(device))).cpu().detach().numpy()),0)
        label = np.concatenate((label,batch[1]),0)

In [16]:
s = 0
for i in range(len(attrs_default)):
    average_precision = roc_auc_score(label[:,i], y_pred[:,i])
    line_new = '{:>15}  {:.4f}'.format(attrs_default[i], average_precision)
    s += average_precision
    print(line_new)
line_new = '{:>15}  {:.4f}'.format("Average:", s/40)
print(line_new)

5_o_Clock_Shadow  0.9742
Arched_Eyebrows  0.9097
     Attractive  0.9188
Bags_Under_Eyes  0.8977
           Bald  0.9953
          Bangs  0.9894
       Big_Lips  0.7474
       Big_Nose  0.8849
     Black_Hair  0.9614
     Blond_Hair  0.9848
         Blurry  0.9528
     Brown_Hair  0.9353
 Bushy_Eyebrows  0.9512
         Chubby  0.9577
    Double_Chin  0.9593
     Eyeglasses  0.9968
         Goatee  0.9899
      Gray_Hair  0.9899
   Heavy_Makeup  0.9774
High_Cheekbones  0.9513
           Male  0.9987
Mouth_Slightly_Open  0.9859
       Mustache  0.9811
    Narrow_Eyes  0.8676
       No_Beard  0.9885
      Oval_Face  0.7540
      Pale_Skin  0.9732
    Pointy_Nose  0.8188
Receding_Hairline  0.9543
    Rosy_Cheeks  0.9593
      Sideburns  0.9901
        Smiling  0.9833
  Straight_Hair  0.8779
      Wavy_Hair  0.9232
Wearing_Earrings  0.9461
    Wearing_Hat  0.9947
Wearing_Lipstick  0.9889
Wearing_Necklace  0.8195
Wearing_Necktie  0.9805
          Young  0.9340
       Average:  0.9411


### Linear Labeling